In [2]:
#Download data file if not done already
#! curl https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt --output ./data/input.txt

In [3]:
#read dataset
with open("./data/input.txt", 'r', encoding='utf-8') as f:
  all_data =  f.read()

print(f"Total length of data : {len(all_data)}")

Total length of data : 1115394


In [4]:
#Build vocab
unique_chars = sorted(list(set(all_data)))
vocab_size = len(unique_chars)

#create mappings between chars and index
ctoi = {ch:id_ for id_, ch in enumerate(unique_chars)}
itoc = {id_:ch for id_, ch in enumerate(unique_chars)}

encode = lambda s: [ctoi[c] for c in s]
decode = lambda ids: "".join([itoc[i] for i in ids])

print(encode("Hello world!!"))
print(decode(encode("Hello world!!")))

[20, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 2, 2]
Hello world!!


In [5]:
#Encode the dataset
import torch
data = torch.tensor(encode(all_data), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [6]:
#split into train and validataion datasets
split_thresh = int(0.9*len(data)) #first 90% as train, remaining 10% as validation
train_data = data[:split_thresh]
val_data = data[split_thresh:]

In [7]:
max_context_length = 8
batch_size = 4

torch.manual_seed(42)

def get_batch(split):
  data = train_data if split=="train" else val_data
  rand_idx = torch.randint(len(data) - max_context_length, (batch_size, ))
  X = torch.stack([data[i:i+max_context_length] for i in rand_idx])
  y = torch.stack([data[i+1:i+max_context_length+1] for i in rand_idx])
  return X, y

#test and sanity check
xb, yb = get_batch("train")
print(f"inputs : {xb.shape}")
print(xb)
print(f"outputs : {yb.shape}")
print(yb)

print("##########################################")

for b in range(batch_size):
  for t in range(max_context_length):
    context = xb[b,:t+1]
    target = yb[b,t]
    print(f"When context is {context}, target is {target}")

inputs : torch.Size([4, 8])
tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [ 1, 58, 46, 43, 56, 43,  1, 41],
        [17, 26, 15, 17, 10,  0, 32, 53],
        [57, 58,  6,  1, 61, 47, 58, 46]])
outputs : torch.Size([4, 8])
tensor([[ 1, 46, 47, 57,  1, 50, 53, 60],
        [58, 46, 43, 56, 43,  1, 41, 39],
        [26, 15, 17, 10,  0, 32, 53,  1],
        [58,  6,  1, 61, 47, 58, 46,  0]])
##########################################
When context is tensor([57]), target is 1
When context is tensor([57,  1]), target is 46
When context is tensor([57,  1, 46]), target is 47
When context is tensor([57,  1, 46, 47]), target is 57
When context is tensor([57,  1, 46, 47, 57]), target is 1
When context is tensor([57,  1, 46, 47, 57,  1]), target is 50
When context is tensor([57,  1, 46, 47, 57,  1, 50]), target is 53
When context is tensor([57,  1, 46, 47, 57,  1, 50, 53]), target is 60
When context is tensor([1]), target is 58
When context is tensor([ 1, 58]), target is 46
When context is ten

In [8]:
#simplest language model: BigramLanguage model

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
      super().__init__()
      self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
  
  def forward(self, idx, targets = None):
      #shape idx: (B, T), shape targets: (B, T)
      logits = self.token_embedding_table(idx) # (B, T, C)

      if targets is None:
        loss = None
      else:
        # tensor conversion from (B, T, C) to (B, C, T) as per cross_entropy documentation
        B, T, C = logits.shape
        logits = logits.view(B*T, C) #reshaped to (B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)
      
      return logits, loss

  def generate(self, idx, max_context_length):
      #idx is of shape: (B, T)
      for _ in range(max_context_length):
        #get predictions
        logits, loss = self(idx)
        #use only the last time step
        logits = logits[:, -1, :]
        probs = F.softmax(logits, dim = -1)
        #sample from the distribution

        idx_next = torch.multinomial(probs, num_samples=1)
        idx = torch.cat((idx, idx_next), dim=1)

      return idx


bgm = BigramLanguageModel(vocab_size)
logits, loss = bgm(xb, yb)

print(f"output shape: {logits.shape}")
print(f"output loss: {loss.item()}")

#test the generator
idx = torch.zeros((1, 1), dtype=torch.long)
print(f"{decode(bgm.generate(idx, max_context_length=100)[0].tolist())}")

output shape: torch.Size([32, 65])
output loss: 4.886534690856934

uoiaF$z
M?kI;h
DbuMG,H3LYNmrDxKgTpvAKOF-jU.hc;fBMTGa-IS
g3lEb&ZQ,l;:m;lpcNN
KpVEYRIIM,'hCRbMAcWTkrnH


In [9]:
#torch oprimization objective
optimizer = torch.optim.AdamW(bgm.parameters(), lr=1e-3)

In [16]:
batch_size = 32
for steps in range(1000):
    
    xb, yb = get_batch("train")

    #evaluate and train
    logits, loss = bgm(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

3.193429946899414


In [18]:
gen_idx =torch.zeros((1, 1), dtype=torch.long)
print(decode(bgm.generate(gen_idx, max_context_length=400)[0].tolist()))


tX;f'czQfffUN mPxn'CUThWor bl;UJsDBEYheOrorrecdyJk-lmisf.xYMr cINusjzGE&,C w,SneceroGPHao in:EPey.wy y!IArg$XWRAnumWk 
MA .
JSjVz,LS bor?YRDul.kneo fuzI-b I.OMm pteaveMAu HPck-K?
QuYy, yZ'

ON.hedsw,laeegee
OPABludonach ylDWhve
'd;iU.c&I;.t


WCu:mak-KY?x3LD,abaqPMn,
Jon&GTOhouUur-e?ac'wO:manUNUp-nt;h b&r'piQz;'KCB: agshhnilwat,
&3TscGpwnttr&INLHXERLJPryNnegid,
Jq pI njbaowhe FfaPF3AVB
yZgh'dJlnt&


# Mathematical Trick in self-attention

In [19]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

## averaging past tokens: basic

In [25]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t, C)
        xbow[b,t] = torch.mean(xprev, 0)

In [28]:
#trick using matrix multiplication
#version 1
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a/torch.sum(a, dim=1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()

c = a @ b

print(f'a= :\n {a}')
print(f'b= :\n {b}')
print(f'c= :\n {c}')

a= :
 tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b= :
 tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c= :
 tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [34]:
#trick using matrix multiplication vectorised
# version 2
wei = torch.tril(torch.ones(T, T))
wei = wei/wei.sum(1, keepdim=True)

xbow2 = wei @ x # (T, T) @ (B, T, C) ---> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [35]:
#version 3 using softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = torch.masked_fill(wei, tril ==0, float('-inf'))
wei = F.softmax(wei, dim=-1)

xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [32]:
#version 4 :single head of self attention
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

head_size = 16
query = nn.Linear(C, head_size, bias=False)
key = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

q = query(x) # (B, T, head_size)
k = key(x) # (B, T, head_size)
v = value(x) # (B, T, head_size)

wei = q @ k.transpose(-2, -1) * head_size**-0.5 #(B, T, head_size) @ (B, head_size, T) ----> (B, T, T), scaled attention to preserve variance in the weights
#tril = torch.tril(torch.ones((T, T)))

#wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

out = wei @ v


In [33]:
out.shape

torch.Size([4, 8, 16])

In [38]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1)
k.var(), q.var(), wei.var()

(tensor(1.0449), tensor(1.0700), tensor(17.4690))

In [39]:
wei = q @ k.transpose(-2, -1) * head_size**-0.5
k.var(), q.var(), wei.var()

(tensor(1.0449), tensor(1.0700), tensor(1.0918))